In [1]:
import keras, os, pickle, re, sklearn, string, tensorflow
import numpy as np
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras.layers import Embedding
from keras.optimizers import Adadelta
from sklearn.model_selection import train_test_split


print('Keras version: \t\t%s' % keras.__version__)
print('Scikit version: \t%s' % sklearn.__version__)
print('TensorFlow version: \t%s' % tensorflow.__version__)


Using TensorFlow backend.


Keras version: 		2.2.4
Scikit version: 	0.21.2
TensorFlow version: 	1.13.1


In [2]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
nltk.download('punkt')   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence,remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')# Download text data sets, including stop words
from nltk.corpus import stopwords # Import the stop word list
import re
from bs4 import BeautifulSoup

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
def review_to_wordlist( review, remove_stopwords=False ):
# Function to convert a document to a sequence of words,
# optionally removing stop words. Returns a list of words.
#
# 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
#
# 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
#
# 3. Convert words to lower case and split them
    words = review_text.lower().split()
#
# 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
#
# 5. Return a list of words
    return(words)


In [19]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
nltk.download('punkt')
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
# Function to split a review into parsed sentences. Returns a
# list of sentences, where each sentence is a list of words
#
# 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
#
# 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
# If a sentence is empty, skip it
        if len(raw_sentence) > 0:
# Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, remove_stopwords ))
#
# Return the list of sentences (each sentence is a list of words,
# so this returns a list of lists
    return sentences


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
def read_files(path):
    documents = list()
    print("11111")
# Read in all files in directory
    if os.path.exists(path):
        print("22")
        for filename in os.listdir(path):
            with open('%s/%s' % (path, filename),encoding='utf-8') as f:

                doc = f.read()
                # doc = clean_text(doc)
                documents.append(doc)
# Read in all lines in a txt file
    if os.path.isfile(path):
        with open(path, encoding='iso-8859-1') as f:
            doc = f.readlines()
            for line in doc:
                documents.append(line)
    return documents


In [99]:
from google.colab import drive
import glob

drive.mount('/content/gdrive/')
!ls "/content/gdrive/My Drive"
download_path = os.path.expanduser("/content/gdrive/My Drive")
try:
  os.makedirs(download_path)
except FileExistsError:
  pass

def read_files(path):
    print(path)

    path = glob.glob('/gdrive/aclImdb/train/pos/')
    print(path)
    documents = list()
    print("11111")
    '''
    for filename in os.listdir(path):
      with open('%s/%s' % (path, filename),encoding='utf-8') as f:
        print("2222")

        doc = f.read()
        documents.append(doc)
        
    if os.path.isfile(path):
        with open(path, encoding='iso-8859-1') as f:
            doc = f.readlines()
            for line in doc:
                documents.append(line)
    return documents
  '''

Mounted at /content/gdrive/
 2016w1.gdoc				      kaggle.json
 49495575-22695143.gdoc			      NLP.gdoc
 aclImdb				      part1.gsheet
 aclImdb.zip				      part2.gsheet
 clouderizer				      part3.gsheet
'cm,all data.gsheet'			      Phrasebook.gform
'Colab Notebooks'			      Phrasebook.gsheet
 CV.gdoc				      plan.gdoc
 data					     'Project Overview.gdoc'
'Getting started.pdf'			      Readme.txt.gdoc
 glove.840B.300d.txt			      test
 H					     'tools_gpa_calculator (1).gsheet'
 hamm1.gsheet				     'tools_gpa_calculator (5).gsheet'
 hfhfh.oxps				      train
'Introduction to TensorFlow(Kadenze).ipynb'


In [106]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
'''
# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass
'''
# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
file_list = drive.ListFile(
    {'q': "'1zMBSNv_KEvwUY-3pj7Pd0HWWo8D0-t08' in parents"}).GetList()

for f in file_list:
  # 3. Create & download by id.
  print('title: %s, id: %s' % (f['title'], f['id']))
  fname = os.path.join(local_download_path, f['title'])
  print('downloading to {}'.format(fname))
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)


with open(fname, 'r') as f:
  print(f.read())
  documents.append(doc)
# Read in all lines in a txt file
  if os.path.isfile(path):
    with open(path, encoding='iso-8859-1') as f:
      doc = f.readlines()
      for line in doc:
        documents.append(line)
        
        
print(documents)

title: 12454_7.txt, id: 1UPuG7A9kghFRVp3SxCUKU_0goby6VCEC
downloading to /root/data/12454_7.txt
title: 12277_10.txt, id: 1WOAbDq8gM07JQSv6M9SbNwOvwaljst8r
downloading to /root/data/12277_10.txt
title: 12262_7.txt, id: 1Wyl-8O_1ON0axyoUQeFACArLd1JjMEl-
downloading to /root/data/12262_7.txt
title: 12352_8.txt, id: 1SctMcyEVQ0fUo1iU9Rb_5yGlXvDfU9IV
downloading to /root/data/12352_8.txt
title: 12343_7.txt, id: 1U6f03UPLMF_D67CwGndzYXvm6pJxtuvS
downloading to /root/data/12343_7.txt
title: 12465_9.txt, id: 1VS4ToZfLYRRlsR0nxo5LJ8WonAxgMy_E
downloading to /root/data/12465_9.txt
title: 12289_9.txt, id: 1VnAWplsQuS75Qv3a6SdXUvPa1X1x1ZBd
downloading to /root/data/12289_9.txt
title: 12445_9.txt, id: 1XVR0pUhhrBLaawXg82ElMFIOqL7tiYCj
downloading to /root/data/12445_9.txt
title: 12368_8.txt, id: 1X_eSnWgG9IccjEUlzSds9BvwgIFxcbYU
downloading to /root/data/12368_8.txt
title: 12356_8.txt, id: 1V-Ben3Utis3GVjbWHekopCMqW5YglW3c
downloading to /root/data/12356_8.txt
title: 12433_8.txt, id: 1VsG1Ruu80_ARh

ApiRequestError: ignored

In [107]:
pos_train = read_files('/content/drive/aclImdb/train/pos')
neg_train = read_files('/content/gdrive/My Drive/aclImdb/train/neg')
pos_test = read_files('/content/gdrive/My Drive/aclImdb/test/pos')
neg_test = read_files('/content/gdrive/My Drive/aclImdb/test/neg')
train1 = pos_train + neg_train
test1 = pos_test + neg_test

#docs = negative_docs + positive_docs
l_train = [1 for _ in range(len(pos_train))] + [0 for _ in range(len(neg_train))]
l_test = [1 for _ in range(len(pos_train))] + [0 for _ in range(len(neg_test))]
train = np.column_stack((train1,l_train))
test = np.column_stack((test1,l_test))

/content/drive/aclImdb/train/pos
[]
11111
/content/gdrive/My Drive/aclImdb/train/neg
[]
11111
/content/gdrive/My Drive/aclImdb/test/pos
[]
11111
/content/gdrive/My Drive/aclImdb/test/neg
[]
11111


TypeError: ignored

In [110]:
print(documents)

NameError: ignored

In [33]:
sentences = [] # Initialize an empty list of sentences
num_reviews = 25000
print ("Parsing sentences from training set")
for review in range( 0, num_reviews ):
    sentences += review_to_sentences(train[review][0], tokenizer)
'''
print ("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)
'''

Parsing sentences from training set


IndexError: ignored

In [0]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',    level=logging.INFO)

def myhashfxn(obj):
    return hash(obj) % (2 ** 32)

# Set values for various parameters
num_features = 400    # Word vector dimensionality                      
min_word_count = 60   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words


#**********************************************************



# Initialize and train the model (this will take some time)
from gensim.models import word2vec
#model = word2vec.Word2Vec(hashfxn=myhashfxn)

print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers,             size=num_features, min_count = min_word_count,             window = context, sample = downsampling,hashfxn=myhashfxn)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)


In [0]:
model.doesnt_match("man woman child kitchen".split())


In [0]:

model.most_similar("man")



In [0]:
#Vector Averaging
import numpy as np # Make sure that numpy is imported
def makeFeatureVec(words, model, num_features):
# Function to average all of the word vectors in a given
# paragraph
#
# Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
#
    nwords = 0.
#
# Index2word is a list that contains the names of the words in
# the model's vocabulary. Convert it to a set, for speed
    index2word_set = set(model.wv.index2word)
#
# Loop over each word in the review and, if it is in the model's
# vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
#
# Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec



In [0]:
def getAvgFeatureVecs(reviews, model, num_features):
# Given a set of reviews (each one a list of words), calculate
# the average feature vector for each one and return a 2D numpy array
#
# Initialize a counter
    counter = 0
#
# Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
# Loop through the reviews
    for review in reviews:
        if counter%1000 == 0:
            print ("Review %d of %d" % (counter, len(reviews)))
        reviewFeatureVecs[counter] = makeFeatureVec(review, model,num_features)
#
# Increment the counter
        counter = counter + 1
    return reviewFeatureVecs




In [0]:
clean_train_reviews = []
for review in train1:
    clean_train_reviews.append( review_to_wordlist( review,         remove_stopwords=True ))
    
trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )
print ("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test1:
    clean_test_reviews.append( review_to_wordlist( review,         remove_stopwords=True ))


# In[52]: